More realistic example of LinearRegression

Format data for processing

In [1]:
from pyspark.sql import SparkSession

In [2]:
spark = SparkSession.builder.appName('linear-regression-continued').getOrCreate()

In [3]:
from pyspark.ml.regression import LinearRegression

In [4]:
data = spark.read.csv("Ecommerce_Customers.csv", inferSchema=True,header=True)

In [5]:
data.printSchema()

root
 |-- Email: string (nullable = true)
 |-- Address: string (nullable = true)
 |-- Avatar: string (nullable = true)
 |-- Avg Session Length: double (nullable = true)
 |-- Time on App: double (nullable = true)
 |-- Time on Website: double (nullable = true)
 |-- Length of Membership: double (nullable = true)
 |-- Yearly Amount Spent: double (nullable = true)



In [6]:
data.show()

+--------------------+--------------------+----------------+------------------+------------------+------------------+--------------------+-------------------+
|               Email|             Address|          Avatar|Avg Session Length|       Time on App|   Time on Website|Length of Membership|Yearly Amount Spent|
+--------------------+--------------------+----------------+------------------+------------------+------------------+--------------------+-------------------+
|mstephenson@ferna...|835 Frank TunnelW...|          Violet| 34.49726772511229| 12.65565114916675| 39.57766801952616|  4.0826206329529615|  587.9510539684005|
|   hduke@hotmail.com|4547 Archer Commo...|       DarkGreen| 31.92627202636016|11.109460728682564|37.268958868297744|    2.66403418213262|  392.2049334443264|
|    pallen@yahoo.com|24645 Valerie Uni...|          Bisque|33.000914755642675|11.330278057777512|37.110597442120856|   4.104543202376424| 487.54750486747207|
|riverarebecca@gma...|1414 David Throug...|   

In [9]:
for item in data.head(1)[0]:
    print(item)

mstephenson@fernandez.com
835 Frank TunnelWrightmouth, MI 82180-9605
Violet
34.49726772511229
12.65565114916675
39.57766801952616
4.0826206329529615
587.9510539684005


Set up dataframe for ML

We want to predict Yearly spend by customer email

Features are `Avg Session Length`, `Time on App`, `Time on Website`, `Length of Membership`

In [10]:
from pyspark.ml.linalg import Vectors

In [11]:
from pyspark.ml.feature import VectorAssembler

In [13]:
data.columns

['Email',
 'Address',
 'Avatar',
 'Avg Session Length',
 'Time on App',
 'Time on Website',
 'Length of Membership',
 'Yearly Amount Spent']

In [14]:
assembler = VectorAssembler(
    inputCols=['Avg Session Length', 'Time on App', 'Time on Website', 'Length of Membership'],
    outputCol='features'
)

In [16]:
output = assembler.transform(data)

In [17]:
output.printSchema()

root
 |-- Email: string (nullable = true)
 |-- Address: string (nullable = true)
 |-- Avatar: string (nullable = true)
 |-- Avg Session Length: double (nullable = true)
 |-- Time on App: double (nullable = true)
 |-- Time on Website: double (nullable = true)
 |-- Length of Membership: double (nullable = true)
 |-- Yearly Amount Spent: double (nullable = true)
 |-- features: vector (nullable = true)



In [18]:
output.head(1)

[Row(Email='mstephenson@fernandez.com', Address='835 Frank TunnelWrightmouth, MI 82180-9605', Avatar='Violet', Avg Session Length=34.49726772511229, Time on App=12.65565114916675, Time on Website=39.57766801952616, Length of Membership=4.0826206329529615, Yearly Amount Spent=587.9510539684005, features=DenseVector([34.4973, 12.6557, 39.5777, 4.0826]))]

In [19]:
final_data = output.select('features', 'Yearly Amount Spent')

In [20]:
final_data.show()

+--------------------+-------------------+
|            features|Yearly Amount Spent|
+--------------------+-------------------+
|[34.4972677251122...|  587.9510539684005|
|[31.9262720263601...|  392.2049334443264|
|[33.0009147556426...| 487.54750486747207|
|[34.3055566297555...|  581.8523440352177|
|[33.3306725236463...|  599.4060920457634|
|[33.8710378793419...|   637.102447915074|
|[32.0215955013870...|  521.5721747578274|
|[32.7391429383803...|  549.9041461052942|
|[33.9877728956856...|  570.2004089636196|
|[31.9365486184489...|  427.1993848953282|
|[33.9925727749537...|  492.6060127179966|
|[33.8793608248049...|  522.3374046069357|
|[29.5324289670579...|  408.6403510726275|
|[33.1903340437226...|  573.4158673313865|
|[32.3879758531538...|  470.4527333009554|
|[30.7377203726281...|  461.7807421962299|
|[32.1253868972878...| 457.84769594494855|
|[32.3388993230671...| 407.70454754954415|
|[32.1878120459321...|  452.3156754800354|
|[32.6178560628234...|   605.061038804892|
+----------

In [22]:
training_data,test_data = final_data.randomSplit([0.7,0.3])

In [24]:
training_data.describe().show()

+-------+-------------------+
|summary|Yearly Amount Spent|
+-------+-------------------+
|  count|                356|
|   mean|  500.6503743441149|
| stddev|   82.6508062431165|
|    min| 256.67058229005585|
|    max|  765.5184619388373|
+-------+-------------------+



In [25]:
test_data.describe().show()

+-------+-------------------+
|summary|Yearly Amount Spent|
+-------+-------------------+
|  count|                144|
|   mean| 496.01031849160114|
| stddev|  70.55517665781558|
|    min|  319.9288698031936|
|    max|  689.7876041747194|
+-------+-------------------+



In [26]:
lr = LinearRegression(featuresCol='features', labelCol='Yearly Amount Spent')

In [27]:
lr_model = lr.fit(training_data)

In [29]:
test_results = lr_model.evaluate(test_data)

In [30]:
test_results.residuals.show()

+-------------------+
|          residuals|
+-------------------+
| -12.57592948292779|
| -4.970374997911165|
| 6.5116276649623615|
|  -21.9574743358354|
| 18.315970861266862|
|  3.638802208602897|
| 0.3609158190229209|
| -5.646699107767631|
|-18.407329563151052|
| 16.346591928231703|
|-26.866644406063756|
| -6.898441094355292|
| -5.256355598762923|
| -19.01050262899463|
| -2.035171742246064|
|  7.869669648457204|
| -5.390568763446879|
|-14.057554158176572|
|  4.442531619211991|
|  8.377205744398111|
+-------------------+
only showing top 20 rows



residuals are the difference between predicted data and actual data

In [31]:
test_results.rootMeanSquaredError

9.89807565176907

In [32]:
test_results.r2

0.9801815253678704

In [33]:
final_data.describe().show()

+-------+-------------------+
|summary|Yearly Amount Spent|
+-------+-------------------+
|  count|                500|
|   mean|  499.3140382585909|
| stddev|   79.3147815497068|
|    min| 256.67058229005585|
|    max|  765.5184619388373|
+-------+-------------------+



Yearly Amount Spent mean of 499.31

Root Mean Square of 9.89
10 error on 500 is pretty good

R2 of 0.980 (98%)
model explains 98% of model

Results this good are suspicious - double check you did not accidentally fit and evaluate data on training data

Deploy data

In [34]:
unlabeld_data = test_data.select('features')

In [37]:
predictions = lr_model.transform(unlabeld_data)

In [38]:
predictions.show()

+--------------------+------------------+
|            features|        prediction|
+--------------------+------------------+
|[30.3931845423455...| 332.5047992861214|
|[30.8794843441274...|495.17697498276584|
|[30.9716756438877...|488.12698209193036|
|[31.1239743499119...| 508.9045281756012|
|[31.3123495994443...|445.27544716667376|
|[31.3662121671876...|  426.950080347882|
|[31.3895854806643...|409.70869524095997|
|[31.5257524169682...| 449.6123259176495|
|[31.5702008293202...| 564.3528217045559|
|[31.6098395733896...|428.19895772287646|
|[31.6739155032749...|502.59171231594496|
|[31.7242025238451...| 510.2863283823158|
|[31.7656188210424...|501.81043723437006|
|[31.8164283341993...|  520.132994132651|
|[31.8186165667690...| 448.4538451123817|
|[31.8512531286083...| 465.1225770183412|
|[31.8745516945853...| 397.6758130097144|
|[31.9365486184489...|441.25693905350477|
|[31.9480174211613...| 457.4783452736858|
|[31.9549038566348...|431.62067419552886|
+--------------------+------------